In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Set environment variables
import os

os.environ["SCAL_TYPE"] = "complex"
os.environ["PRECISION"] = "double"
os.environ["MY_NUMBA_TARGET"] = "cuda"
 
# Add cle_fun to PYTHON_PATH
import sys
sys.path.append("../../clonscal")

In [2]:
from src.numba_target import myjit

@myjit
def test_cuda_act_kernel(idx, act_matrix, array, val):
    array[idx] = val

Using CUDA


/home/fhelmberger/miniconda3/envs/cle_cuda/lib/python3.10/site-packages/llvmlite/llvmpy/__init__.py:3: UserWarning: The module `llvmlite.llvmpy` is deprecated and will be removed in the future.
  warnings.warn(
/home/fhelmberger/miniconda3/envs/cle_cuda/lib/python3.10/site-packages/llvmlite/llvmpy/core.py:8: UserWarning: The module `llvmlite.llvmpy.core` is deprecated and will be removed in the future. Equivalent functionality is provided by `llvmlite.ir`.
  warnings.warn(
/home/fhelmberger/miniconda3/envs/cle_cuda/lib/python3.10/site-packages/llvmlite/llvmpy/passes.py:17: UserWarning: The module `llvmlite.llvmpy.passes` is deprecated and will be removed in the future. If you are using this code, it should be inlined into your own project.
  warnings.warn(


In [3]:
from numba import cuda

size = int(1e7)

array = np.zeros(size)
activation = np.random.choice(a=[False, True], size=size, p=[0.5, 0.5])

d_array = cuda.to_device(array)
d_activation = cuda.to_device(activation)

In [4]:
from src.numba_target import my_act_parallel_loop, use_cuda

if not use_cuda:
    my_act_parallel_loop(test_cuda_act_kernel, size, activation, array, 1)
    print(os.environ["MY_NUMBA_TARGET"]+" mode:")
    %timeit my_act_parallel_loop(test_cuda_act_kernel, size, activation, array, 1)
else:
    print(os.environ["MY_NUMBA_TARGET"]+" mode:")
    my_act_parallel_loop(test_cuda_act_kernel, size, d_activation, d_array, 1)
    %timeit my_act_parallel_loop(test_cuda_act_kernel, size, d_activation, d_array, 1)

python mode:
1.23 s ± 19.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
from src.numba_target import my_act_parallel_loop, use_cuda

if not use_cuda:
    my_act_parallel_loop(test_cuda_act_kernel, size, activation, array, 1)
    print(os.environ["MY_NUMBA_TARGET"]+" mode:")
    %timeit my_act_parallel_loop(test_cuda_act_kernel, size, activation, array, 1)
else:
    print(os.environ["MY_NUMBA_TARGET"]+" mode:")
    my_act_parallel_loop(test_cuda_act_kernel, size, d_activation, d_array, 1)
    %timeit my_act_parallel_loop(test_cuda_act_kernel, size, d_activation, d_array, 1)

numba mode:
2.14 ms ± 45.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [4]:
from src.numba_target import my_act_parallel_loop, use_cuda

if not use_cuda:
    my_act_parallel_loop(test_cuda_act_kernel, size, activation, array, 1)
    print(os.environ["MY_NUMBA_TARGET"]+" mode:")
    %timeit my_act_parallel_loop(test_cuda_act_kernel, size, activation, array, 1)
else:
    print(os.environ["MY_NUMBA_TARGET"]+" mode:")
    my_act_parallel_loop(test_cuda_act_kernel, size, d_activation, d_array, 1)
    %timeit my_act_parallel_loop(test_cuda_act_kernel, size, d_activation, d_array, 1)

cuda mode:
169 µs ± 9.15 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
